In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

import librosa
from IPython.display import Audio

In [2]:
!wget -r https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/4R8jAFyoX4Edzg

--2024-06-27 14:43:03--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/4R8jAFyoX4Edzg
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/30c52c846095489be934f8303eafae53f57249ab54c71e8cf1a64eb4a9853b9e/667db2b9/fKqInKw3d7bLFOeFnMGnhPZNAvgT-oS2GAaEPHDZGJj_CHQFbzdyKYOnc9yGfg5uYyrQ1ZEWjFgzmmGyWwplsIA_OwEc-VcLL3KrPxnz2Der8npumZHI4midPdWhecNq?uid=0&filename=voice_ger.zip&disposition=attachment&hash=UsYArs01gKfycQJo4QrZuclQCeRaPynH2oi%2BqnL5oFb2oQXvuPOrYI0eLq/WKK0WDqZvSgIch5AN9ddz7ydViQ%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=1130000063874175&fsize=32051680&hid=5a4717eae4b5a75b554f8b304939845d&media_type=compressed&tknv=v2 [following]
--2024-06-27 14:43:05--  https://downloader.disk.yandex.ru/disk/30c52c846095489be934f8303eafae53f57249ab54c71e8cf1a64eb4a9

In [3]:
!unzip -u /content/getfile.dokpub.com/yandex/get/https:/disk.yandex.ru/d/4R8jAFyoX4Edzg

Archive:  /content/getfile.dokpub.com/yandex/get/https:/disk.yandex.ru/d/4R8jAFyoX4Edzg
   creating: voice_ger/
   creating: voice_ger/02/
  inflating: voice_ger/02/0_02_0.wav  
  inflating: voice_ger/02/0_02_1.wav  
  inflating: voice_ger/02/0_02_10.wav  
  inflating: voice_ger/02/0_02_11.wav  
  inflating: voice_ger/02/0_02_12.wav  
  inflating: voice_ger/02/0_02_13.wav  
  inflating: voice_ger/02/0_02_14.wav  
  inflating: voice_ger/02/0_02_15.wav  
  inflating: voice_ger/02/0_02_16.wav  
  inflating: voice_ger/02/0_02_17.wav  
  inflating: voice_ger/02/0_02_18.wav  
  inflating: voice_ger/02/0_02_19.wav  
  inflating: voice_ger/02/0_02_2.wav  
  inflating: voice_ger/02/0_02_20.wav  
  inflating: voice_ger/02/0_02_21.wav  
  inflating: voice_ger/02/0_02_22.wav  
  inflating: voice_ger/02/0_02_23.wav  
  inflating: voice_ger/02/0_02_24.wav  
  inflating: voice_ger/02/0_02_25.wav  
  inflating: voice_ger/02/0_02_26.wav  
  inflating: voice_ger/02/0_02_27.wav  
  inflating: voice_ger/0

In [4]:
person=2
digit = 0

In [5]:
#Modified get_data()
def get_data_m(digit=0,person=person,index=0,target_sr=16000):
    if person < 10:
        file = f"/content/voice_ger/0{person}/{digit}_0{person}_{index}.wav"
    else:
        file = f"/content/voice_ger/{person}/{digit}_{person}_{index}.wav"

    data, sr = librosa.load(file)#sr=22050
    #down sampling to 8000Hz
    down_d = librosa.resample(data, orig_sr=sr, target_sr=target_sr)
    #fixed length of all data to 12000 samples
    fix_len_d = librosa.util.fix_length(down_d, size=12000)
    return fix_len_d, target_sr

In [6]:
def read_data(digit=0, person=2, index=0):
    data_per = get_data_m(digit=digit, person=person, index=index)
    return data_per

In [7]:
df_0 = pd.DataFrame()
df_1 = pd.DataFrame()

In [ ]:
for i in range(0,50):
    df_0[i] = pd.DataFrame(read_data(digit=0, index=i, person=2)[0])

In [ ]:
for i in range(0,50):
    df_1[i] = pd.DataFrame(read_data(digit=1, index=i, person=2)[0])

In [ ]:
df_0 = df_0.add_suffix('_0')
df_1 = df_1.add_suffix('_1')

In [ ]:
df = pd.concat([df_0, df_1], axis =1)

In [ ]:
target = [0 if x<50 else 1 for x in range(0,100)]

In [ ]:
np.unique(target, return_counts= True)

Давайте обучим модель БЕЗ извлечения признаков

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
X = df.T.values

In [ ]:
Y = np.array(target)[None].T

In [ ]:
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=.3,
                                                 shuffle=True,random_state=42)
print(X_train.shape,Y_train.shape, X_test.shape,Y_test.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=10)
clf.fit(X_train, Y_train)
print("Accuracy on training set is : {:.2f}".format(clf.score(X_train, Y_train)))
print("Accuracy on test set is : {:.2f}".format(clf.score(X_test, Y_test)))
Y_test_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_test_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_test_pred)

Давайте теперь обучим модель на основе выделенных признаков

In [ ]:
!pip install tsfel

In [ ]:
import tsfel

In [ ]:
cfg_file = tsfel.get_features_by_domain('spectral') # 'temporal', 'statistical'

fs=1000 # частота дискретизации


In [ ]:
X_train_tsfel = pd.DataFrame()

for i in range(0,len(X_train)):
    X_train_tsfel[i] = tsfel.time_series_features_extractor(cfg_file, X_train[i], fs=fs).T

In [ ]:
X_test_tsfel = pd.DataFrame()

for i in range(0,len(X_test)):
    X_test_tsfel[i] = tsfel.time_series_features_extractor(cfg_file, X_test[i], fs=fs).T

In [ ]:
X_train.shape

In [ ]:
X_train_tsfel.T.values

In [ ]:
clf = RandomForestClassifier(n_estimators=3,max_depth=4)
clf.fit(X_train_tsfel.T.values, Y_train)
print("Accuracy on training set is : {:.2f}".format(clf.score(X_train_tsfel.values.T, Y_train)))
print("Accuracy on test set is : {:.2f}".format(clf.score(X_test_tsfel.T.values, Y_test)))
Y_test_pred = clf.predict(X_test_tsfel.T.values)
print(classification_report(Y_test, Y_test_pred))

Вывод: при более примитивных характеристиках классификатора, использование tsfel позволяет повысить точностные характеристики, выделив наиболее существенные признаки из набора данных